In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import requests        
import json           
import pandas as pd    
import numpy as np    
import matplotlib.pyplot as plt 
import datetime as dt  

/home/pedro/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:47: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# from backtesting.test import SMA, GOOG
# GOOG.to_csv("data/GOOG.csv")

In [10]:
def get_bars(symbol, interval = '1h'):
   root_url = 'https://api.binance.com/api/v1/klines'
   url = root_url + '?symbol=' + symbol + '&interval=' + interval
   data = json.loads(requests.get(url).text)
   
   columns = ['open_time',
                 'Open', 'High', 'Low', 'Close', 'Volume',
                 'close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore']
   dtypes = ['int64','int64', 'int64', 'int64', 'int64', 'int64','int64', 'int64', 'int64','int64', 'int64', 'int64']
   df = pd.DataFrame(data,columns=columns)
   df.index = [dt.datetime.fromtimestamp(x/1000.0).date() for x in df.close_time]
   df.sort_values(["open_time"],ascending=False)
   df = df.astype('double')
   df['Volume'] = df['Volume'].astype('int')
   df = df.drop(['open_time','close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore'],axis=1)
   df.to_csv("data/"+symbol+"_"+interval+".csv", sep=';')
   return df

In [11]:
btcusd = get_bars("BTCUSDT", "1d")

In [12]:
class SmaCross(Strategy):
    n1=0
    n2=0

        
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            
    def setInitialParams(self,valor1, valor2):
        self.n1 = valor1;
        self.n2 = valor2;

In [6]:
maiorRetorno = {}
maoirAcerto = {}
tamanho = 3
for s1 in range(tamanho):
    for s2 in range(tamanho):
#         print("testando nos valores ", s1, " e ", s2)
        SmaCross.setInitialParams(SmaCross,s1, s2)
        bt = Backtest(btcusd, SmaCross,cash=100000000,exclusive_orders=True)
        output = bt.run()
        if maiorRetorno=={}:
            maiorRetorno = {"sma1": s1, "sma2":s2, "output":output}
        if maoirAcerto=={}:
            maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
        if output['Return [%]'] > maiorRetorno["output"]['Return [%]'] and output['# Trades'] > 9:
            maiorRetorno = {"sma1": s1, "sma2":s2, "output":output}
        if output['Win Rate [%]'] > maiorRetorno["output"]['Win Rate [%]'] and output['# Trades'] > 9:
            maoirAcerto = {"sma1": s1, "sma2":s2, "output":output}
            
# print(maiorRetorno)
# print(maoirAcerto)
        

testando nos valores  0  e  0
testando nos valores  0  e  1
testando nos valores  0  e  2
testando nos valores  1  e  0
testando nos valores  1  e  1
testando nos valores  1  e  2
testando nos valores  2  e  0
testando nos valores  2  e  1
testando nos valores  2  e  2
{'sma1': 2, 'sma2': 1, 'output': Start                     2020-06-11 20:59:59
End                       2021-10-23 20:59:59
Duration                    499 days 00:00:00
Exposure Time [%]                        99.2
Equity Final [$]                  2.38337e+08
Equity Peak [$]                   3.11165e+08
Return [%]                            138.337
Buy & Hold Return [%]                 559.886
Return (Ann.) [%]                     88.5167
Volatility (Ann.) [%]                  166.79
Sharpe Ratio                         0.530708
Sortino Ratio                         1.71182
Calmar Ratio                          1.28725
Max. Drawdown [%]                    -68.7644
Avg. Drawdown [%]                    -11.7214
Max. Dr

In [13]:
# SmaCross.setInitialParams(SmaCross,12, 15)
# bt = Backtest(GOOG, SmaCross,cash=1000,exclusive_orders=True)
# output = bt.run()
# print(output)
SmaCross.setInitialParams(SmaCross,28, 29)
bt = Backtest(btcusd, SmaCross,cash=1000000)
output = bt.run()
output
bt.plot()

/home/pedro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  


Row(id='1645', ...)